<a href="https://colab.research.google.com/github/abay-qkt/madb-exploration/blob/main/%E9%80%B1%E5%88%8A%E5%B0%91%E5%B9%B4%E6%BC%AB%E7%94%BB%E9%9B%91%E8%AA%8C%E3%81%AE%E4%BE%A1%E6%A0%BC%E3%81%AE%E6%8E%A8%E7%A7%BB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 準備

In [1]:
!pip install sparqlwrapper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.8 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import plotly.express as px

In [3]:
from SPARQLWrapper import SPARQLWrapper

# エンドポイントの設定
sparql = SPARQLWrapper(endpoint='https://mediaarts-db.artmuseums.go.jp/sparql', returnFormat='json')

# クエリを投げてDataFrameを返す関数
def read_sparql(query):
  sparql.setQuery(query)
  response = sparql.queryAndConvert()  # 辞書形式で結果を受け取る
  df = pd.DataFrame(response["results"]["bindings"]).applymap(lambda x:x["value"])  # DataFrameに変換する
  df = df[response["head"]["vars"]]  # 列の並び替え
  return df

# 各マンガ雑誌の値段の推移

In [4]:
query_price = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <https://schema.org/>
SELECT distinct ?label ?price ?number_of_pages ?total_volume_number ?date_published ?name ?one_magazine
WHERE {
  ?one_magazine schema:isPartOf ?magazine ;
  schema:price ?price ;
  rdfs:label ?label ;
  schema:numberOfPages ?number_of_pages ;
  <https://mediaarts-db.bunka.go.jp/data/property#totalVolumeNumber> ?total_volume_number ;
  schema:datePublished ?date_published ;
  schema:name ?name .
  VALUES ?magazine {
    <https://mediaarts-db.bunka.go.jp/id/C119459> # ジャンプ
    <https://mediaarts-db.bunka.go.jp/id/C119033> # マガジン
    <https://mediaarts-db.bunka.go.jp/id/C117607> # サンデー
    <https://mediaarts-db.bunka.go.jp/id/C120282> # チャンピオン
  }
  FILTER(LANG(?name) != "ja-hrkt")  # nameはフリガナも返していたので除外
}
"""

In [5]:
df_price = read_sparql(query_price)

In [6]:
df_price.head(5)

,label,price,number_of_pages,total_volume_number,date_published,name,one_magazine
0,週刊少年サンデー 2013年 表示号数18,257円+税,490,3163,2013-04-03,週刊少年サンデー,https://mediaarts-db.bunka.go.jp/id/M616843
1,週刊少年サンデー 2013年 表示号数41,257円+税,514,3189,2013-09-11,週刊少年サンデー,https://mediaarts-db.bunka.go.jp/id/M616864
2,週刊少年サンデー 2014年 表示号数27,250円+税,496,3233,2014-06-04,週刊少年サンデー,https://mediaarts-db.bunka.go.jp/id/M616899
3,週刊少年サンデー 2014年 表示号数17,257円+税,496,3222,2014-03-26,週刊少年サンデー,https://mediaarts-db.bunka.go.jp/id/M616890
4,週刊少年サンデー 2014年 表示号数10,257円+税,500,3213,2014-02-05,週刊少年サンデー,https://mediaarts-db.bunka.go.jp/id/M616883


In [7]:
df_price["total_volume_number"] = df_price["total_volume_number"].astype(int)
df_price["date_published"] = pd.to_datetime(df_price["date_published"])
df_price["price"] = df_price["price"].str.extract(r'(\d+)').astype(float)
df_price["number_of_pages"] = df_price["number_of_pages"].str.extract(r'(\d+)').astype(float).astype(int)

In [8]:
df_price.head(5)

,label,price,number_of_pages,total_volume_number,date_published,name,one_magazine
0,週刊少年サンデー 2013年 表示号数18,257.0,490,3163,2013-04-03,週刊少年サンデー,https://mediaarts-db.bunka.go.jp/id/M616843
1,週刊少年サンデー 2013年 表示号数41,257.0,514,3189,2013-09-11,週刊少年サンデー,https://mediaarts-db.bunka.go.jp/id/M616864
2,週刊少年サンデー 2014年 表示号数27,250.0,496,3233,2014-06-04,週刊少年サンデー,https://mediaarts-db.bunka.go.jp/id/M616899
3,週刊少年サンデー 2014年 表示号数17,257.0,496,3222,2014-03-26,週刊少年サンデー,https://mediaarts-db.bunka.go.jp/id/M616890
4,週刊少年サンデー 2014年 表示号数10,257.0,500,3213,2014-02-05,週刊少年サンデー,https://mediaarts-db.bunka.go.jp/id/M616883


In [9]:
fig = px.line(df_price.sort_values('date_published'),x='date_published',y='price',color='name',hover_data=["label"])
fig.update_layout(hovermode="x")